In [2]:
import openeo

In [3]:
connection = openeo.connect("openeo.dataspace.copernicus.eu").authenticate_oidc()

Visit https://identity.dataspace.copernicus.eu/auth/realms/CDSE/device?user_code=ZTVO-MSOZ 📋 to authenticate.

✅ Authorized successfully

Authenticated using device code flow.


### 1. Create a datacube for period during COVID lockdowns

In [20]:
aoi_fc = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "geometry": {
                "type": "Polygon",
                "coordinates": [
                    [
                        [77.11, 28.69],
                        [77.11, 28.56],
                        [77.29, 28.56],
                        [77.29, 28.69],
                        [77.11, 28.69]
                    ]
                ]
            },
            "properties": {}
        }
    ]
}

In [21]:
s5covid = connection.load_collection(
    "SENTINEL_5P_L2",
    temporal_extent=["2020-06-01", "2021-06-30"],
    spatial_extent={"west": 77.11, "south": 28.56, "east": 77.29, "north": 28.69},
    bands=["NO2"],
)

In [22]:
# Now aggregate by day to avoid having multiple data per day
s5covid = s5covid.aggregate_temporal_period(reducer="mean", period="month")

# let's create a spatial aggregation to generate mean timeseries data
s5covid = s5covid.aggregate_spatial(reducer="mean", geometries=aoi_fc)

### 2. Let’s repeat the same process for Post Covid Situation

In [23]:
# Create a datacube for period after COVID lockdowns

s5post = connection.load_collection(
    "SENTINEL_5P_L2",
    temporal_extent=["2022-06-01", "2023-06-30"],
    spatial_extent={"west": 77.11, "south": 28.56, "east": 77.29, "north": 28.69},
    bands=["NO2"],
)

# Now aggregate by day to avoid having multiple data per day
s5post = s5post.aggregate_temporal_period(reducer="mean", period="month")

# Now create a spatial aggregation to generate mean timeseries data
s5post = s5post.aggregate_spatial(reducer="mean", geometries=aoi_fc)

Finally we execute them as batch jobs and download the results as netCDF. They are further plotted as shown below to study the data.

In [24]:
job = s5covid.execute_batch(title="NO2 during Covid", outputfile="during_covid.nc")

0:00:00 Job 'j-25112904295744fab8caf60e9d91335e': send 'start'
0:00:12 Job 'j-25112904295744fab8caf60e9d91335e': created (progress 0%)
0:00:18 Job 'j-25112904295744fab8caf60e9d91335e': queued (progress 0%)
0:00:24 Job 'j-25112904295744fab8caf60e9d91335e': queued (progress 0%)
0:00:32 Job 'j-25112904295744fab8caf60e9d91335e': queued (progress 0%)
0:00:42 Job 'j-25112904295744fab8caf60e9d91335e': queued (progress 0%)
0:00:54 Job 'j-25112904295744fab8caf60e9d91335e': queued (progress 0%)
0:01:10 Job 'j-25112904295744fab8caf60e9d91335e': queued (progress 0%)
0:01:29 Job 'j-25112904295744fab8caf60e9d91335e': running (progress N/A)
0:01:53 Job 'j-25112904295744fab8caf60e9d91335e': running (progress N/A)
0:02:23 Job 'j-25112904295744fab8caf60e9d91335e': running (progress N/A)
0:03:00 Job 'j-25112904295744fab8caf60e9d91335e': running (progress N/A)
0:03:47 Job 'j-25112904295744fab8caf60e9d91335e': running (progress N/A)
0:04:45 Job 'j-25112904295744fab8caf60e9d91335e': error (progress N/A)
You

JobFailedException: Batch job 'j-25112904295744fab8caf60e9d91335e' didn't finish successfully. Status: error (after 0:04:45).

In [ ]:
job = s5post.execute_batch(title="NO2 Post-Covid", outputfile="post_covid.nc")

### Let’s plot the result

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt

In [ ]:
# load the results
duringdata = xr.load_dataset("during_covid.nc")
postdata = xr.load_dataset("post_covid.nc")

Let’s calculate the mean along-time dimension for the window size of 30days.

In [ ]:
duringdata = duringdata.rolling(t=30).mean()
postdata = postdata.rolling(t=30).mean()

In [ ]:
fig, ax1 = plt.subplots(dpi=100)
(line1,) = ax1.plot(
    duringdata.t, duringdata.NO2.to_numpy().flatten(), color="r", label="During COVID"
)
ax1.set_xlabel("During COVID")
ax1.set_ylabel("NO2 Level")
ax1.xaxis.label.set_color("r")
ax1.tick_params(axis="x", colors="r")

ax2 = ax1.twiny()
(line2,) = ax2.plot(
    postdata.t, postdata.NO2.to_numpy().flatten(), color="g", label="Post COVID"
)
ax2.set_xlabel("Post COVID")
# Combine legends from both axes
lines = [line1, line2]
labels = [line.get_label() for line in lines]
ax1.legend(lines, labels, loc="upper left")
ax2.xaxis.label.set_color("g")
ax2.tick_params(axis="x", colors="g")